In [10]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

# Item 1.1 - Preparação dos dados

In [2]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [26]:
# Junta as instancias dos dois conjuntos em um unico dataset
data = np.concatenate((X_train, X_test))

# Junta os targets dos dois conjuntos em um unico vetor
target = np.concatenate((y_train, y_test))

In [28]:
print(data.shape, target.shape, sep='\n')

(70000, 28, 28)
(70000,)


### Define as proporções dos cortes

In [49]:
cut_10 = int(data.shape[0] * 0.1)  # 10%
cut_20 = int(data.shape[0] * 0.2)  # 20%
cut_30 = int(data.shape[0] * 0.3)  # 30%
cut_40 = int(data.shape[0] * 0.4)  # 40%
cut_50 = int(data.shape[0] * 0.5)  # 50%
cut_60 = int(data.shape[0] * 0.6)  # 60%
cut_70 = int(data.shape[0] * 0.7)  # 70%
cut_80 = int(data.shape[0] * 0.8)  # 80%
cut_90 = int(data.shape[0] * 0.9)  # 90%

### Realiza os cortes

In [50]:
# Corte em 10%
X_10 = data[:cut_10]
y_10 = target[:cut_10]

# Corte em 20%
X_20 = data[:cut_20]
y_20 = target[:cut_20]

# Corte em 30%
X_30 = data[:cut_30]
y_30 = target[:cut_30]

# Corte em 40%
X_40 = data[:cut_40]
y_40 = target[:cut_40]

# Corte em 50%
X_50 = data[:cut_50]
y_50 = target[:cut_50]

# Corte em 60%
X_60 = data[:cut_60]
y_60 = target[:cut_60]

# Corte em 70%
X_70 = data[:cut_70]
y_70 = target[:cut_70]

# Corte em 80%
X_80 = data[:cut_80]
y_80 = target[:cut_80]

# Corte em 90%
X_90 = data[:cut_90]
y_90 = target[:cut_90]

# Corte em 100% (nao realiza corte)
X_100 = data
y_100 = target